In [ ]:
import tensorflow
from tensorflow.keras.applications import VGG16 # importing VGG16 model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # for Data Augumentation

In [ ]:
batchsize=64
path_train='/content/drive/MyDrive/sign_mobilenet/data/train'
path_test='/content/drive/MyDrive/sign_mobilenet/data/test'

In [ ]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.3,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2,horizontal_flip=True, validation_split=1/9)

train_data= train_datagen.flow_from_directory(path_train,
                                target_size=(224,224),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(path_train,
                                target_size=(224,224),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 677 images belonging to 1 classes.
Found 84 images belonging to 1 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(path_test,
                                target_size=(224,224),batch_size=batchsize,class_mode='categorical')

Found 79 images belonging to 1 classes.


In [ ]:
bmodel = VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=(224,224,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(1024, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'sigmoid')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [ ]:
checkpoint = ModelCheckpoint('model_.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=20, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=20, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=100)

<ipython-input-31-7fdaac157157>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/100
10/10 [==============================] - ETA: 0s - loss: 400.1974 - accuracy: 0.6962 
Epoch 1: val_loss did not improve from 60.25803
10/10 [==============================] - 156s 15s/step - loss: 400.1974 - accuracy: 0.6962 - val_loss: 109.0472 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - ETA: 0s - loss: 445.6084 - accuracy: 0.6000 
Epoch 2: val_loss did not improve from 60.25803
10/10 [==============================] - 164s 17s/step - loss: 445.6084 - accuracy: 0.6000 - val_loss: 189.0809 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - ETA: 0s - loss: 445.2487 - accuracy: 0.6689 
Epoch 3: val_loss did not improve from 60.25803
10/10 [==============================] - 153s 15s/step - loss: 445.2487 - accuracy: 0.6689 - val_loss: 175.5736 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - ETA: 0s - loss: 469.3392 - accuracy: 0.6250 
Epoch 4: val_loss di